# Logistic Regression Interpretation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For our modeling steps
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative purposes
from scipy.special import logit, expit
from sklearn import datasets

# Objectives

- Explain the form of logistic regression
- Explain how to interpret logistic regression coefficients
- Use logistic regression to perform a classification task

# Cost Functions and Solutions To the Optimization Problem

Unlike the least-squares problem for linear regression, no one has yet found a closed-form solution to the optimization problem presented by logistic regression. But even if one exists, the computation would no doubt be so complex that we'd be better off using some sort of approximation method instead.

But there's still a problem.

Recall the cost function for linear regression: <br/><br/>
$SSE = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i(y_i - (\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in}))^2$.

This function, $SSE(\vec{\beta})$, is convex.

If we plug in our new logistic equation for $\hat{y}$, we get: <br/><br/>
$SSE_{log} = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i\left(y_i - \left(\frac{1}{1+e^{-(\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in})}}\right)\right)^2$.

## The Bad News

*This* function, $SSE_{log}(\vec{\beta})$, is [**not** convex](https://towardsdatascience.com/why-not-mse-as-a-loss-function-for-logistic-regression-589816b5e03c).

That means that, if we tried to use gradient descent or some other approximation method that looks for the minimum of this function, we could easily find a local rather than a global minimum.

> Note that the scikit-learn class *expects the user to specify the solver* to be used in calculating the coefficients. The default solver, [lbfgs](https://en.wikipedia.org/wiki/Limited-memory_BFGS), works well for many applications.

## The Good News

We can use **log-loss** instead:

$\mathcal{L}(\vec{y}, \hat{\vec{y}}) = -\frac{1}{N}\Sigma^N_{i=1}\left(y_iln(\hat{y}_i)+(1-y_i)ln(1-\hat{y}_i)\right)$,

where $\hat{y}_i$ is the probability that $(x_{i1}, ... , x_{in})$ belongs to **class 1**.

**More resources on the log-loss function**:

https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11

https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a

### 🧠 Knowledge Check

- Is a bigger value (more positive) better or worse than a smaller value?

-

# Digging Deeper into Logistic Regression

## Preparing Data

In [ ]:
# glass identification dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
col_names = ['id','ri','na','mg','al','si','k','ca','ba','fe','glass_type']
glass = pd.read_csv(url, names=col_names, index_col='id')
glass.sort_values('al', inplace=True)
glass.head()

In [ ]:
# types 1, 2, 3 are window glass
# types 5, 6, 7 are household glass
glass['household'] = glass.glass_type.map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head()

## Train Logistic Regression

In [ ]:
# fit a logistic regression model and store the class predictions

logreg = LogisticRegression(random_state=42)
feature_cols = ['al']
X = glass[feature_cols]
y = glass.household
logreg.fit(X, y)
glass['household_pred_class'] = logreg.predict(X)

## Log-Loss

In [ ]:
log_loss(y, logreg.predict_proba(X))

### Verifying by Hand

In [ ]:
y_hat = logreg.predict_proba(X)[:, 1]

In [ ]:
-np.mean([y*np.log(y_hat) + (1-y)*np.log(1-y_hat)])

## Interpreting Logistic Regression Coefficients

In [ ]:
logreg.coef_

How do we interpret the coefficients of a logistic regression? For a linear regression, the situaton was like this:

- Linear Regression: We construct the best-fit line and get a set of coefficients. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change in $y$.

- Logistic Regression: We find the coefficients of the best-fit line by some approximation method. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change (not in $y$ but) in $ln\left(\frac{y}{1-y}\right)$.

We have:

$\ln\left(\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}\right) = \ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k$.

Exponentiating both sides:

$\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)} = e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k}$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right)}\cdot e^k$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^k\cdot\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}$

That is, the odds ratio at $x_1+1$ has increased by a factor of $e^k$ relative to the odds ratio at $x_1$.

For more on interpretation, see [this page](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/how-to/binary-logistic-regression/interpret-the-results/all-statistics-and-graphs/coefficients/).

In [ ]:
# examine the intercept

logreg.intercept_

**Interpretation:** For an 'al' value of 0, the log-odds of 'household' is -6.01. What is the probability that glass with an 'al' value of 0 is household glass?

In [ ]:
# convert log-odds to probability

logodds = logreg.intercept_
odds = np.exp(logodds)
odds

In [ ]:
prob = odds / (1 + odds)
prob

In [ ]:
expit(logodds)

In [ ]:
# examine the coefficient for al

list(zip(feature_cols, logreg.coef_[0]))

**Interpretation:** A 1 unit increase in 'al' is associated with a 3.12-unit increase in the log-odds of 'household'.

##### Aside: Verifying log-odds to probability

Let's verify this as we change the aluminum content from 1 to 2.

In [ ]:
# Prediction for al=1

pred_a1 = logreg.predict_proba([[1]])
pred_a1

In [ ]:
pred_a1[0][1]

In [ ]:
# Odds ratio for al=1

odds_a1 = pred_a1[0][1] / pred_a1[0][0]
odds_a1

In [ ]:
# Prediction for al=2

pred_a2 = logreg.predict_proba([[2]])
pred_a2

In [ ]:
# Odds ratio for al=2

odds_a2 = pred_a2[0][1] / pred_a2[0][0]
odds_a2

In [ ]:
(np.exp(logreg.coef_[0]) * odds_a1)[0]

In [ ]:
np.exp(logreg.coef_[0])

##### Aside: Use Coefficients to Generate Prediction

In [ ]:
# compute predicted log-odds for al=2 using the equation
x_al = 2
logodds = logreg.intercept_ + logreg.coef_[0] * x_al
logodds

In [ ]:
# convert log-odds to odds

odds = np.exp(logodds)
odds

In [ ]:
# convert odds to probability

prob = odds / (1 + odds)
prob

In [ ]:
# compute predicted probability for al=2 using the predict_proba method

logreg.predict_proba(np.array([2.0]).reshape(1, 1))[:, 1]

# Putting It All Together: Training Logistic Regression

Let's take some time to show how you can do use logistic regression in practice.

> Note we've been talking about _binary classification_ but we can also do classification for _multiclass_ problems (more than binary classes).
>
> That's what we'll do for this example.

## Load and Explore Data

In [ ]:
# Built in dataset from sklearn
iris = datasets.load_iris()

df = pd.DataFrame(
    data=np.c_[iris['data'], iris['target']],
    columns=iris['feature_names'] + ['target']
)

display(df.head())
display(df.describe())

In [ ]:
df.info()

In [ ]:
# Note how many different targets there are
df.target.unique()

In [ ]:
df['target'].value_counts()

We can go ahead and explore some graphs to show that it doesn't make sense to do a linear regression

In [ ]:
# Creating a large figure
fig = plt.figure(figsize=(15, 8))

# Iterating over the different
for i in range(0, 4):
    # Figure number starts at 1
    ax = fig.add_subplot(2, 2, i+1)
    # Add a title to make it clear what each subplot shows
    plt.title(df.columns[i])
    # Use alpha to better see crossing pints
    ax.scatter(df['target'], df.iloc[:, i], c='teal', alpha=0.1)
    # Only show the tick marks for each target
    plt.xticks(df.target.unique())

Preparing the data for classification

In [ ]:
# Get the features and then the target
X = df.drop('target', axis=1)
y = df.target

In [ ]:
# Normalize the data to help the model
X = normalize(X)
X

In [ ]:
# Now each row vector has unit length
sum([x**2 for x in X[0, :]])

In [ ]:
# Split for test & training  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

## Create and Train Logistic Regression Model

There's a lot of different parameters for `LogisticRegression`. Check out the documentation for more info: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
my_model = LogisticRegression(
            C=1000,             # Smaller values -> more regularization
            max_iter= 100,      # Ensure we eventually reach a solution
            solver='lbfgs',    # (Default) Can optimize depending on problem
            multi_class='ovr', # (Default) Will try to do multiclass classification 
            random_state=42)

In [ ]:
# Fit/Train the model
my_model.fit(X_train, y_train)
my_model

### Evaluate the Model with Cross-Validation

In practice, we should make this a practice but we skip it if time is running low

In [ ]:
cv_results = cross_validate(estimator=my_model, X=X_train, y=y_train, cv=5,
                            return_train_score=True)

In [ ]:
cv_results['train_score'].mean()

In [ ]:
cv_results['test_score'].mean()

In [ ]:
def cv_overall(cv_results):
    val_results = cv_results['test_score']
    result_str = f'{val_results.mean():.3f} ± {val_results.std():.3f}'
    return result_str

In [ ]:
cv_overall(cv_results)

In [ ]:
# Let's save these results for later
models = {}

models['model_1'] = {'model': my_model, 'cv':cv_results}

In [ ]:
# Overall Training Score
my_model.fit(X_train, y_train)
my_model.score(X_train,y_train)

### Rinse and Repeat - Multiple Models

Let's try out a few more models for fun

In [ ]:
# Adjust the regularization C
c_values = [1e-1, 1e2, 1e4, 1e6]

# Start at #2 since we have "model_1" already
for i, c in enumerate(c_values, start=2):
    
    print(f'Model #{i} with C={c}')
    new_model = LogisticRegression(C=c, max_iter=1e3, random_state=42)
    
    # Cross-validation
    print('Cross-validating model with training data...')
    cv_results = cross_validate(
                    estimator=new_model,
                    X=X_train,
                    y=y_train,
                    cv=5,
                    return_train_score=True
    )
    print(f'\tCross-Validation Score: {cv_overall(cv_results)}')
    
    # Train/fit with the full training set
    print('Fitting model to full training set...')
    new_model.fit(X_train, y_train)
    train_score = new_model.score(X_train, y_train)
    print(f'\tScore on training set: {train_score:.3f}')
    
    # Save results
    print('Saving Results...')
    models[f'model_{i}'] = {'model': new_model, 'cv': cv_results}
    
    print('\n','-'*30,'\n')

## Final Evaluation

### END GOAL OF PREDICTIVE MODELLING IS THE MODEL THAT PERFORMS THE BEST ON 'UNSEEN' DATA (aka test or validation)

In [ ]:
models

In [ ]:
best_model = models['model_5']['model']
best_model

In [ ]:
# Let's get predictions for training & testing sets
y_hat_train = best_model.predict(X_train) 
y_hat_test = best_model.predict(X_test)

### Log-Loss on Best Model

In [ ]:
log_loss(y_train, y_hat_train)

In [ ]:
log_loss(y_train, best_model.predict_proba(X_train))

In [ ]:
log_loss(y_test, best_model.predict_proba(X_test))

### Training Set

In [ ]:
# Was our model correct?
residuals = y_train == y_hat_train

print('Number of values correctly predicted:')
print(pd.Series(residuals).value_counts())

print('\n','-'*30,'\n')

print('Percentage of values correctly predicted: ')
print(pd.Series(residuals).value_counts(normalize=True))

### Testing Set

In [ ]:
residuals = y_test == y_hat_test

print('Number of values correctly predicted:')
print(pd.Series(residuals).value_counts())

print('\n','-'*30,'\n')

print('Percentage of values correctly predicted: ')
print(pd.Series(residuals).value_counts(normalize=True))

# Exercise

Split the data below into train and test, and then convert the y-values (`geysers.kind`) into 1's and 0's. Then use `sklearn` to build a logistic regression model of whether Old Faithful's eruption wait time is long or short, based on the duration of the eruption. Finally, find the points in the test set where the model's prediction differs from the true y-value. How many are there?

In [ ]:
geysers = sns.load_dataset('geyser', **{'usecols': ['duration', 'kind']})

In [ ]:
geysers.head()

# Level Up

## More Generalizations: Other Link Functions, Other Models

Logistic regression's link function is the logit function, but different sorts of models use different link functions.

[Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) has a nice table of generalized linear model types and their associated link functions.